In [1]:
#install transformers from huggingface

! git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [2]:
! cd transformers && git reset --hard 143738214

HEAD is now at 143738214 Fix the loss calculation of ProphetNet (#13132)


In [3]:
# Install NLTK

! pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.8 MB/s eta 0:00:0000:0100:010m
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 681.9 kB/s eta 0:00:00a 0:00:01
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 15.9 MB/s eta 0:00:0000:01
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)


In [19]:
# Install Tokenizers

! pip install tokenizers

In [5]:
# Install sentencepiece
! pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.0 MB/s eta 0:00:0000:0100:010m


In [3]:
# Import necessary Libraries
import os
import pandas as pd
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import tensorflow as tf
from transformers import T5Tokenizer, T5ForConditionalGeneration


# Initialize our model
tokenizer = T5Tokenizer.from_pretrained("orzhan/t5-long-extract")
model = T5ForConditionalGeneration.from_pretrained("orzhan/t5-long-extract")


[nltk_data] Downloading package punkt to /home/student/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/student/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-01-31 20:59:36.942151: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-31 20:59:37.316871: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-31 20:59:37.319823: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-31 20:59:38.163899: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
You are using the default lega

In [2]:
def read_file(filenameWithPath:str)->str:
  '''
  This function reads the content of a file.

  Parameters:
  filenameWithPath : str
    Absolute Path of the file which is to be read.

  Returns:
  text : str
    The content of the read file. 

  '''
  file = open(filenameWithPath,"r")
  text = file.read().strip()
  text = re.sub("\n"," ",text)
  return text

In [3]:
def preprocessing(text):
  '''
  The preprocessing function which parses raw text and manages to clean the raw text. This module removes websites/URLs, Email IDs, redundant spaces, extra periods, dashes and commas. It also replace some special unicode apostrope, qoutation symbols with ' and  ". This module also ads extra space after period, question mark and exclamation mark.

  Parameters:
    text : str
      The raw text to preprocess.

  Returns:
    text : str
      The preprocessed text. 
  '''
  # text = text.lower()
  text = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " ", text)
  text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', " ",text)
  text = re.sub("\u2019", '\'', text)
  text = re.sub("\u2018", '\'', text)
  text = re.sub("\u201C", '\"', text)
  text = re.sub("\u201D", '\"', text)
  # text = re.sub("[$]", "dollar ", text)
  # text = re.sub("[£]", "pound ", text)
  # text = re.sub("[%]", " percent", text)
  # text = re.sub(r"[^a-zA-Z0-9?!.,’-]", ' ', text)
  text = re.sub(r"([?!])", r" \1 ", text)
  text = re.sub(r',+', ',', text)
  text = re.sub(r'[-]+', ' ', text)
  text = re.sub(r'([a-z])(?=[.,])', r'\1 ', text)
  text = re.sub(r'\.{2,}', '.', text)
  text = re.sub(r"\s+", " ", text)
  return text

In [76]:
tokenizer.max_len_single_sentence

511

In [4]:
def createSentences(text):
    '''
    This function uses the nltk library to sentence tokenize a given corpus or document.

    Parameters:
    text : 
        The corpus or document string to split into sentences.

    Returns: 
        A sentence tokenized copy of *text* using NLTK's default tokenizer. A list of sentences.

    '''
    return nltk.tokenize.sent_tokenize(text)

In [5]:
def createChunks(sentences):
  '''
  This module splits the entire corpus into chunks. A chunk is a list of sentences from the corpus. This module add a sentence to a chunk if the tokenized sentence doesn't exceed the models' single sentence length. Else it adds the senence to next chunk.

  Parameters:
  sentences : list
    A list of containing sentences.

  Returns:
  chunks : list
    A list of containing the chunks of the document.
  '''
  length = 0
  chunk = ""
  chunks = []
  count = -1
  for sentence in sentences:
    # print("Sent ", sentence)
    count += 1
    combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

    if combined_length  < tokenizer.max_len_single_sentence - 3 : # if it doesn't exceed
      chunk += sentence + " " # add the sentence to the chunk
      length = combined_length # update the length counter
      # print("length",length)
      # if it is the last sentence
      if count == len(sentences) - 1:
        chunks.append(chunk.strip()) # save the chunk
      # print("if", chunks)
    else:
      chunks.append(chunk.strip()) # save the chunk
      # print("else", chunks)

      # reset
      length = 0
      chunk = ""

      # take care of the overflow sentence
      chunk += sentence + " "
      length = len(tokenizer.tokenize(sentence))
  return chunks

In [6]:
def generate_chunk_summary(chunk):
    '''
    This module generates the summary for a given document. Prior to passing the document to model, it appends 'summarize: ' prefix to document to let the model know that the task is summarization. It restricts the generated summary to 100 tokens.

    Parameters:
    chunk : str
        A chunk is group of sentences such that the tokenized lenght of chunk doesn't exceed the token limit of model.

    Returns:
    summary : str
        It is the summary which model generates for the given text chunk. 
    '''
    input_ids = tokenizer.encode("summarize: " + chunk, return_tensors="pt",max_length=tokenizer.model_max_length, truncation=True)
    summary_ids = model.generate(input_ids,max_length=100)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [7]:
def generate_summary(fileChunks):
    '''
    This function returns the complete summary for a chunked document or corpus of text.

    Parameters:
    fileChunks: list
        A list of chunk where chunk is group of sentences in a corpus such that tokenized length of chunk doesn't exceed the model limit.

    Returns:
        Concatenated Summary as *str* for every chunk. 
    '''
    summaries = []

    for chunk in fileChunks:
        summ = generate_chunk_summary(chunk)
        summaries.append(summ)

    return " ".join(summaries)

In [8]:
! mkdir ./generated_summary

In [ ]:
def cap_summary(generated_summary:str)->str:
    '''
    This function caps the generated summary to 1000 words. 

    Parameters:
    generated_summary : str
        System generated summary.

    Returns:
        Returns Summary as str after capping.
    '''
    sentences = nltk.sent_tokenize(generated_summary)
    new_summary=[]
    target_word=1250
    words_added=0
    for sentence in sentences:
        words= nltk.word_tokenize(sentence)
        if words_added + len(words) <= target_word:
            words_added+=len(words)
            new_summary.append(sentence)
        else:
            break

    return "".join(new_summary)

In [9]:
def summarize():
    '''
    This function generates the summary for all the documents in *data-dir* . It generates the summary or the document and writes the summary in *generated_summary* folder. The convention for written file is *filename_sysSumm.txt*
    '''
    data_dir = "./fns2020_dataset/validation/annual_reports/"
    for file in os.listdir(data_dir):
        
        fileContent = read_file(os.path.join(data_dir,file))
        filePreprocessed = preprocessing(fileContent)
        fileSentences = createSentences(filePreprocessed)
        fileChunks = createChunks(fileSentences)
        summary = generate_summary(fileChunks)
        filename = file[:-4]

        with open(f"./generated_summary/{filename}_sysSumm.txt","w+") as f:
            f.write(summary)
        


summarize()

In [7]:
data_dir = "./generated_summary/"
target_dir= "./capped_summary/"

for file in os.listdir(data_dir):
    filename = os.path.basename(os.path.join(data_dir,file))
    generated_summary = open(os.path.join(data_dir,file),"r").read().strip()

    capped_summary = cap_summary(generated_summary)

    with open(os.path.join(target_dir,filename),"w") as f:
        f.write(capped_summary)

    